# Assignment 2
In this task a cancer cell detection system will be created.

Luke Mason - s3630120
Shekhar Hazari

#### Import packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler

#### Initialise helper functions¶

#### Mount Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

#### Load the main dataset
This data contains both the cell type and isCancerous labels.

In [ ]:
raw_dataframe_main = pd.read_csv('./Image_classification_data/data_labels_maindata.csv', delimiter=',')
raw_dataframe_main.describe()

#### Load the extra dataset
This dataset does not contain the cellType labels.

In [ ]:
raw_dataframe_extra = pd.read_csv('./Image_classification_data/data_labels_extraData.csv', delimiter=',')
raw_dataframe_extra.describe()

In [ ]:
train_df_main, test_df_main = train_test_split(raw_dataframe_main, test_size=0.2, random_state=47)
train_df_main, val_df_main = train_test_split(train_df_main, test_size=0.25, random_state=47)

print("Train data : {}, Val Data: {}, Test Data: {}".format(train_data.shape[0], val_data.shape[0], test_data.shape[0]))

train_df_main['Class'] = train_data['Class'].astype('str')
val_df_main['Class'] = val_data['Class'].astype('str')

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')
val_datagen = ImageDataGenerator(rescale=1./255, data_format='channels_last')

batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_data,
        directory='./',
        x_col="ImgPath",
        y_col="Class",
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_dataframe(
        dataframe=val_data,
        directory='./',
        x_col="ImgPath",
        y_col="Class",
        target_size=(32, 32),
        batch_size=batch_size,
        class_mode='categorical')